In [ ]:
import planetary_computer as pc
from pystac_client import Client
import stackstac
import numpy as np
import rasterio
import xarray as xr
from datetime import datetime, timedelta
import os

# ---------------- PARAMETRY ----------------
# BBOX = [19.03, 49.69, 19.15, 49.78]
BBOX = [19.5, 49.562222, 19.562222, 49.583333]
EPSG = 32634
RESOLUTION = 20
MAX_CLOUD = 30
NDSI_THRESHOLD = 0.4

START_YEAR = 2015
END_YEAR = datetime.now().year

OUTPUT_DIR = "./ndii_daily_babia"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# -------------------------------------------

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)

def daterange(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

# --- PĘTLA PO SEZONACH ---
for season_year in range(START_YEAR, END_YEAR):
    print(f"\n❄️ Sezon {season_year}/{season_year+1}")

    start_date = datetime(season_year, 11, 1)
    end_date = datetime(season_year + 1, 2, 28)

    for day in daterange(start_date, end_date):

        date_str = day.strftime("%Y-%m-%d")
        print(f" ▶ {date_str}")

        search = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=BBOX,
            datetime=f"{date_str}/{date_str}",
            query={"eo:cloud_cover": {"lt": MAX_CLOUD}},
        )

        items = list(search.items())
        if not items:
            print("   ⚠️ Brak danych")
            continue

        # --- STACK DLA JEDNEGO DNIA ---
        stack = stackstac.stack(
            items,
            assets=["B08", "B11"],
            resolution=RESOLUTION,
            bounds_latlon=BBOX,
            epsg=EPSG,
            fill_value=np.nan,
        )

        nir = stack.sel(band="B08")
        swir = stack.sel(band="B11")

        ndii = (nir - swir) / (nir + swir)
        ndii = ndii.where(~np.isinf(ndii))

        # --- JEŚLI WIELE SCEN TEGO DNIA → MEDIANA ---
        ndii_daily = ndii.median(dim="time", skipna=True).values

        output_path = f"{OUTPUT_DIR}/ndii_{date_str}.nc"

        # --- ZAPIS DO XARRAY / NETCDF ---
        da = xr.DataArray(
            ndii_daily.astype(np.float32),
            dims=("y", "x"),
            name="ndii",
            attrs={
                "description": "Daily median NDII",
                "ndii_threshold": NDSI_THRESHOLD,
                "source": "Sentinel-2 L2A",
                "date": date_str,
            },
        )

        ds = xr.Dataset(
            {
                "ndii": da,
                "snow_mask": (da > NDSI_THRESHOLD).astype("uint8"),
            },
            attrs={
                "bbox": BBOX,
                "epsg": EPSG,
                "resolution": RESOLUTION,
            },
        )

        ds.to_netcdf(
            output_path,
            encoding={
                "ndii": {"_FillValue": -9999.0, "dtype": "float32"},
                "snow_mask": {"_FillValue": 0, "dtype": "uint8"},
            },
        )

        print(f"   ✅ zapisano {output_path}")
import os
import numpy as np
import rasterio
from glob import glob
import xarray as xr
import numpy as np

# ---------------- PARAMETRY ----------------
NDSI_DIR = "./ndsi_daily_babia"
NDII_DIR = "./ndii_daily_babia"
OUTPUT_DIR = "./ndii_masked_snow_babia"
NDSI_THRESHOLD = 0.4  # Próg uznania piksela za śnieg
NODATA_VALUE = -9999

os.makedirs(OUTPUT_DIR, exist_ok=True)


# ------------------------------------------

def mask_moisture_by_snow():
    # Pobieramy listę wszystkich plików NDSI
    ndsi_files = glob(os.path.join(NDSI_DIR, "ndsi_*.tif"))

    print(f"🔍 Znaleziono {len(ndsi_files)} plików NDSI. Rozpoczynam maskowanie...")

    for ndsi_path in ndsi_files:
        # Wyciągamy datę z nazwy pliku (np. '2023-12-05')
        filename = os.path.basename(ndsi_path)
        date_str = filename.replace("ndsi_", "").replace(".tif", "")

        # Budujemy ścieżkę do odpowiadającego mu pliku NDII
        ndii_path = os.path.join(NDII_DIR, f"ndii_{date_str}.tif")

        if not os.path.exists(ndii_path):
            print(f"⚠️ Pominiecie: Brak pliku NDII dla daty {date_str}")
            continue

        # Otwieramy oba pliki
        with rasterio.open(ndsi_path) as src_ndsi, rasterio.open(ndii_path) as src_ndii:
            ndsi_data = src_ndsi.read(1)
            ndii_data = src_ndii.read(1)

            # Pobieramy metadane z pliku NDII (do zapisu)
            meta = src_ndii.meta.copy()

            # Tworzymy maskę:
            # Warunek 1: NDSI musi być większe niż próg
            # Warunek 2: Omijamy oryginalne wartości NoData (jeśli istnieją)
            snow_mask = (ndsi_data >= NDSI_THRESHOLD) & (ndsi_data != src_ndsi.nodata)

            # Nakładamy maskę: tam gdzie nie ma śniegu, wstawiamy NODATA_VALUE
            masked_ndii = np.where(snow_mask, ndii_data, NODATA_VALUE)

            # Aktualizujemy metadane dla nowego pliku
            meta.update({
                "nodata": NODATA_VALUE,
                "dtype": "float32"
            })

            # Zapisujemy wynik
            output_path = os.path.join(OUTPUT_DIR, f"ndii_snow_only_{date_str}.tif")
            with rasterio.open(output_path, "w", **meta) as dst:
                dst.write(masked_ndii.astype(np.float32), 1)

            print(
                f"✅ Przetworzono: {date_str} (Śnieg wykryty na {(snow_mask.sum() / snow_mask.size) * 100:.1f}% obszaru)")


if __name__ == "__main__":
    mask_moisture_by_snow()


❄️ Sezon 2015/2016
 ▶ 2015-11-01
   ⚠️ Brak danych
 ▶ 2015-11-02
   ⚠️ Brak danych
 ▶ 2015-11-03
   ⚠️ Brak danych
 ▶ 2015-11-04
   ⚠️ Brak danych
 ▶ 2015-11-05
   ⚠️ Brak danych
 ▶ 2015-11-06
   ⚠️ Brak danych
 ▶ 2015-11-07
   ⚠️ Brak danych
 ▶ 2015-11-08
   ⚠️ Brak danych
 ▶ 2015-11-09
   ⚠️ Brak danych
 ▶ 2015-11-10
   ⚠️ Brak danych
 ▶ 2015-11-11
   ⚠️ Brak danych
 ▶ 2015-11-12
   ⚠️ Brak danych
 ▶ 2015-11-13
   ⚠️ Brak danych
 ▶ 2015-11-14
   ⚠️ Brak danych
 ▶ 2015-11-15
   ⚠️ Brak danych
 ▶ 2015-11-16
   ⚠️ Brak danych
 ▶ 2015-11-17
   ⚠️ Brak danych
 ▶ 2015-11-18
   ⚠️ Brak danych
 ▶ 2015-11-19
   ⚠️ Brak danych
 ▶ 2015-11-20
   ⚠️ Brak danych
 ▶ 2015-11-21
   ⚠️ Brak danych
 ▶ 2015-11-22
   ⚠️ Brak danych
 ▶ 2015-11-23
   ⚠️ Brak danych
 ▶ 2015-11-24
   ⚠️ Brak danych
 ▶ 2015-11-25


In [ ]:
import os
import numpy as np
import rasterio
from glob import glob
import xarray as xr
import numpy as np

# ---------------- PARAMETRY ----------------
NDSI_DIR = "./ndsi_daily_szczyrk"
NDII_DIR = "./ndii_daily_szczyrk"
OUTPUT_DIR = "./ndii_masked_snow"
NDSI_THRESHOLD = 0.4  # Próg uznania piksela za śnieg
NODATA_VALUE = -9999

os.makedirs(OUTPUT_DIR, exist_ok=True)
# ------------------------------------------

def mask_moisture_by_snow():
    # Pobieramy listę wszystkich plików NDSI
    ndsi_files = glob(os.path.join(NDSI_DIR, "ndsi_*.tif"))
    
    print(f"🔍 Znaleziono {len(ndsi_files)} plików NDSI. Rozpoczynam maskowanie...")

    for ndsi_path in ndsi_files:
        # Wyciągamy datę z nazwy pliku (np. '2023-12-05')
        filename = os.path.basename(ndsi_path)
        date_str = filename.replace("ndsi_", "").replace(".tif", "")
        
        # Budujemy ścieżkę do odpowiadającego mu pliku NDII
        ndii_path = os.path.join(NDII_DIR, f"ndii_{date_str}.tif")
        
        if not os.path.exists(ndii_path):
            print(f"⚠️ Pominiecie: Brak pliku NDII dla daty {date_str}")
            continue

        # Otwieramy oba pliki
        with rasterio.open(ndsi_path) as src_ndsi, rasterio.open(ndii_path) as src_ndii:
            ndsi_data = src_ndsi.read(1)
            ndii_data = src_ndii.read(1)
            
            # Pobieramy metadane z pliku NDII (do zapisu)
            meta = src_ndii.meta.copy()
            
            # Tworzymy maskę: 
            # Warunek 1: NDSI musi być większe niż próg
            # Warunek 2: Omijamy oryginalne wartości NoData (jeśli istnieją)
            snow_mask = (ndsi_data >= NDSI_THRESHOLD) & (ndsi_data != src_ndsi.nodata)
            
            # Nakładamy maskę: tam gdzie nie ma śniegu, wstawiamy NODATA_VALUE
            masked_ndii = np.where(snow_mask, ndii_data, NODATA_VALUE)
            
            # Aktualizujemy metadane dla nowego pliku
            meta.update({
                "nodata": NODATA_VALUE,
                "dtype": "float32"
            })

            # Zapisujemy wynik
            output_path = os.path.join(OUTPUT_DIR, f"ndii_snow_only_{date_str}.tif")
            with rasterio.open(output_path, "w", **meta) as dst:
                dst.write(masked_ndii.astype(np.float32), 1)
            
            print(f"✅ Przetworzono: {date_str} (Śnieg wykryty na {(snow_mask.sum()/snow_mask.size)*100:.1f}% obszaru)")

if __name__ == "__main__":
    mask_moisture_by_snow()

🔍 Znaleziono 71 plików NDSI. Rozpoczynam maskowanie...
✅ Przetworzono: 2019-12-06 (Śnieg wykryty na 29.7% obszaru)
✅ Przetworzono: 2025-01-18 (Śnieg wykryty na 28.1% obszaru)
✅ Przetworzono: 2020-11-08 (Śnieg wykryty na 0.0% obszaru)
✅ Przetworzono: 2019-11-24 (Śnieg wykryty na 0.0% obszaru)
✅ Przetworzono: 2020-11-25 (Śnieg wykryty na 5.0% obszaru)
✅ Przetworzono: 2022-01-09 (Śnieg wykryty na 43.0% obszaru)
✅ Przetworzono: 2023-01-02 (Śnieg wykryty na 0.4% obszaru)
✅ Przetworzono: 2021-12-03 (Śnieg wykryty na 5.2% obszaru)
✅ Przetworzono: 2020-02-09 (Śnieg wykryty na 60.6% obszaru)
✅ Przetworzono: 2020-01-25 (Śnieg wykryty na 41.8% obszaru)
✅ Przetworzono: 2019-11-01 (Śnieg wykryty na 0.0% obszaru)
✅ Przetworzono: 2020-01-08 (Śnieg wykryty na 51.7% obszaru)
✅ Przetworzono: 2018-11-11 (Śnieg wykryty na 0.0% obszaru)
✅ Przetworzono: 2017-01-08 (Śnieg wykryty na 65.9% obszaru)
✅ Przetworzono: 2021-11-03 (Śnieg wykryty na 0.0% obszaru)
✅ Przetworzono: 2021-11-25 (Śnieg wykryty na 4.7% obs

In [9]:
import os
import argparse
from glob import glob
import numpy as np
import xarray as xr

# Domyślne katalogi (możesz nadpisać w CLI / notebooku)
DEFAULT_NDSI = "./ndsi_daily_szczyrk"
DEFAULT_NDII = "./ndii_daily_szczyrk"
DEFAULT_OUT = "./ndsi_ndii_combined"
DEFAULT_THRESHOLD = 0.4

def _auto_dir(default_dir: str, pattern: str):
    # Jeśli domyślny katalog jest pusty lub go brak, spróbuj znaleźć pierwszy plik wzorca w podkatalogach.
    files_here = sorted(glob(os.path.join(default_dir, pattern))) if os.path.isdir(default_dir) else []
    if files_here:
        return default_dir
    fallback = glob(os.path.join("**", pattern), recursive=True)
    if fallback:
        return os.path.dirname(os.path.abspath(fallback[0]))
    return default_dir

parser = argparse.ArgumentParser(description="Połącz NDII i NDSI NetCDF dla wspólnych dat.")
parser.add_argument("ndsi_dir", nargs="?", default=DEFAULT_NDSI, help="Folder z plikami ndsi_*.nc")
parser.add_argument("ndii_dir", nargs="?", default=DEFAULT_NDII, help="Folder z plikami ndii_*.nc")
parser.add_argument("out_dir", nargs="?", default=DEFAULT_OUT, help="Folder wyjściowy na ndsi_ndii_*.nc")
parser.add_argument("--threshold", type=float, default=DEFAULT_THRESHOLD, help="Próg śniegu (NDSI)")
args, _ = parser.parse_known_args()

ndsi_dir = _auto_dir(args.ndsi_dir, "ndsi_*.nc")
ndii_dir = _auto_dir(args.ndii_dir, "ndii_*.nc")
print(f"Używam NDSI z: {ndsi_dir}")
print(f"Używam NDII z: {ndii_dir}")

os.makedirs(args.out_dir, exist_ok=True)
ndii_files = sorted(glob(os.path.join(ndii_dir, "ndii_*.nc")))
print(f"🔗 Łączenie {len(ndii_files)} plików NDII z NDSI")

for ndii_path in ndii_files:
    date_str = os.path.basename(ndii_path).replace("ndii_", "").replace(".nc", "")
    ndsi_path = os.path.join(ndsi_dir, f"ndsi_{date_str}.nc")

    if not os.path.exists(ndsi_path):
        print(f"⚠️ Pominiecie: brak NDSI dla {date_str}")
        continue

    ds_ndii = xr.open_dataset(ndii_path)
    ds_ndsi = xr.open_dataset(ndsi_path)

    ndii_da = ds_ndii.get("ndii", ds_ndii[list(ds_ndii.data_vars)[0]])
    ndsi_da = ds_ndsi.get("ndsi", ds_ndsi[list(ds_ndsi.data_vars)[0]])

    if ndii_da.shape != ndsi_da.shape:
        print(f"⚠️ Rozmiary niezgodne dla {date_str} (ndii {ndii_da.shape} vs ndsi {ndsi_da.shape})")
        ds_ndii.close(); ds_ndsi.close()
        continue

    snow_mask = (ndsi_da > args.threshold).astype("uint8")

    combined = xr.Dataset(
        {
            "ndii": ndii_da.astype(np.float32),
            "ndsi": ndsi_da.astype(np.float32),
            "snow_mask": snow_mask,
        },
        attrs={
            "bbox": ds_ndii.attrs.get("bbox"),
            "epsg": ds_ndii.attrs.get("epsg"),
            "resolution": ds_ndii.attrs.get("resolution"),
            "date": date_str,
        },
    )

    out_path = os.path.join(args.out_dir, f"ndsi_ndii_{date_str}.nc")
    combined.to_netcdf(
        out_path,
        encoding={
            "ndii": {"_FillValue": -9999.0, "dtype": "float32"},
            "ndsi": {"_FillValue": -9999.0, "dtype": "float32"},
            "snow_mask": {"_FillValue": 0, "dtype": "uint8"},
        },
    )

    print(f"✅ Połączono: {out_path}")

    ds_ndii.close()
    ds_ndsi.close()


Używam NDSI z: /home/shadeform/BITEHACK/ML/ndsi_daily_szczyrk
Używam NDII z: ./ndii_daily_szczyrk
🔗 Łączenie 71 plików NDII z NDSI
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2015-11-25.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2016-12-16.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-01-08.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-01-28.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-02-14.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-02-27.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-11-04.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-11-16.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2017-12-26.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2018-02-19.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2018-11-06.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2018-11-11.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2018-11-29.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2019-01-30.nc
✅ Połączono: ./ndsi_ndii_combined/ndsi_ndii_2019-02-07.nc